In [17]:
!pip3 install -U memory_profiler
%load_ext memory_profiler


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [56]:
print(ord("R") & 0b111)

data=[(10,'oo'), (9,'pp')]
i, data = zip(*data)
print(f"{i} and {data}")
data=['A','A','A','A','B','B','A','A']
grp=groupby(data, lambda x:x)
for k, v in grp:
    print(k)
    print(list(v))
    


2
(10, 9) and ('oo', 'pp')
A
['A', 'A', 'A', 'A']
B
['B', 'B']
A
['A', 'A']


In [4]:
def fibonacci_list(num_items):
    numbers = []
    a,b = 0, 1
    while len(numbers) < num_items:
        numbers.append(a)
        a, b = b, a + b
    return numbers
    

In [5]:
def fibonacci_gen(num_items):
    a,b = 0, 1
    while num_items:
        yield a
        a, b = b, a + b
        num_items -= 1


In [6]:
def test_fibonacci_list():
    for i in fibonacci_list(100_000):
        pass


In [7]:
def test_fibonacci_gen():
    for i in fibonacci_gen(100_000):
        pass

In [8]:
%timeit test_fibonacci_list()

191 ms ± 4.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%memit test_fibonacci_list()

peak memory: 477.48 MiB, increment: 401.00 MiB


In [19]:

%timeit test_fibonacci_gen()

123 ms ± 354 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%memit test_fibonacci_gen()

peak memory: 147.73 MiB, increment: 0.03 MiB


# iterator for infinite series


In [21]:
def fibonacci_native():
    i, j = 0, 1
    count = 0
    while j <= 5000:
        if j % 2:
            count += 1
        i, j = j, i + j
    return count

In [27]:
def fibonacci():
    i, j = 0, 1
    while True:
        yield i
        i, j = j, i + j

In [24]:
def fibonacci_transform():
   
    count = 0
    for f in fibonacci():
        if f > 5000:
            break
        if f % 2:
            count += 1
    
    return count

In [36]:
from itertools import takewhile
def fibonacci_itertools():
    first_5000 = takewhile(lambda x: x <= 5000, 
                           fibonacci())
    return sum(1 for x in first_5000 if x % 2)

In [30]:
print(fibonacci_itertools())

<class 'itertools.takewhile'>
13


In [32]:
print(fibonacci_transform())

13


In [34]:
%timeit fibonacci_transform()
%memit fibonacci_transform()

1.11 µs ± 9.18 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
peak memory: 103.58 MiB, increment: 0.06 MiB


In [37]:
%timeit fibonacci_itertools()
%memit fibonacci_itertools()

1.83 µs ± 7.09 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
peak memory: 139.64 MiB, increment: 0.00 MiB


In [38]:
%timeit fibonacci_native()
%memit fibonacci_native()

622 ns ± 2.14 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
peak memory: 139.64 MiB, increment: 0.00 MiB


# Lazy generator evaluation


# islice            allows slicing a potentially infinite generator

# chain            chains together multiple generators

# takewhile     Adds a condition that will end a generator

# cycle             Makes a finite generator infinite by constantly repeating it

# an example using generators to analyze a large dataset: temporal data
   with 20 years , each line is pwr second, time, value, over 631,152,000 data points. do simple anomaly detection,, using generator to save memory

In [38]:
from random import normalvariate, randint
from itertools import count
from datetime import datetime

In [39]:
def read_data(filename):
    with open(filename) as fd:
        for line in fd:
            data = line.strip().split(',')
            timestamp, value = map(int, data)
            yield datetime.fromtimestamp(timestamp), value

In [46]:
def read_fake_data(filename, window_size=7 * 60 * 60 * 24, loops=10):
    weekminutes=7 * 60 * 60 * 24
    window_data=[]
    for timestamp in count():
        # once a week insert an anomalous data point
        if randint(0,  weekminutes-1 ) == 1:
            value = 100
            #value = normalvariate(0,1)
            print(f"the generated timestamp {dayv} and value: {value}")
        else:
            value = normalvariate(0,1)
            #value = 100
        dayv = datetime.fromtimestamp(timestamp)
        #if (value != 100):
        #  print(f"the generated timestamp {dayv} and value: {value}")
        window_data.append((dayv, value))
        window_size -=1
        if window_size == 0:
            yield window_data
            window_data=[]
            loops -=1
            window_size = weekminutes
        if loops == 0:
            break
          
       
        

In [47]:
from itertools import groupby

def groupby_day(iterable):
    key = lambda row: row[0].day     ## the timestamp property day.
    print(f"in groupby_day the iterable")
    for day, data_group in groupby(iterable, key):
        dataList = list(data_group)
        #print(f"groupby_day  key day is: {day} the datalist size: {len(dataList)}")
        #print(f"{dataList[0]} {dataList[1]}")
        
        yield dataList

In [48]:
from scipy.stats import normaltest
from itertools import filterfalse

def is_normal(data, threshold=1e-3):
    #print(f"in is_normal {len(data)}  the data :{data}")  #data still has datetime in the tuple
    _, values = zip(*data)              #this will separate the datetime and value to two list
    k2, p_vale = normaltest(values)
    if p_vale < threshold:
        return False
    else:
        return True

In [49]:
def filter_anomalous_groups(data):
    yield from filterfalse(is_normal, data)

In [50]:


def filter_anomalous_data(data):
    data_group = groupby_day(data)
    yield from filter_anomalous_groups(data_group)

In [51]:
from itertools import islice

   
filename="testdata.txt"
for window_data in read_fake_data(filename):
    data_list = list(window_data)
    #print(f"data from simulated: {len(data_list)}")
    #day, value = data_list[0]
    #print(f"the day is {day} and value is {value}")
    anomaly_generator = filter_anomalous_data(data_list)
    first_five_anomalies = islice(anomaly_generator, 5)

    for data_anomaly in first_five_anomalies:
        start_date = data_anomaly[0][0]
        end_date = data_anomaly[-1][0]
        print(f"Anomaly from {start_date} - {end_date}")

the generated timestamp 1970-01-03 22:13:28 and value: 100
in groupby_day the iterable
Anomaly from 1970-01-03 00:00:00 - 1970-01-03 23:59:59
the generated timestamp 1970-01-09 14:15:03 and value: 100
in groupby_day the iterable
Anomaly from 1970-01-09 00:00:00 - 1970-01-09 23:59:59
the generated timestamp 1970-01-19 10:25:02 and value: 100
in groupby_day the iterable
Anomaly from 1970-01-19 00:00:00 - 1970-01-19 23:59:59
in groupby_day the iterable
the generated timestamp 1970-01-30 00:24:11 and value: 100
in groupby_day the iterable
Anomaly from 1970-01-30 00:00:00 - 1970-01-30 23:59:59
the generated timestamp 1970-02-07 16:50:20 and value: 100
the generated timestamp 1970-02-08 02:34:13 and value: 100
in groupby_day the iterable
Anomaly from 1970-02-07 00:00:00 - 1970-02-07 23:59:59
Anomaly from 1970-02-08 00:00:00 - 1970-02-08 23:59:59
in groupby_day the iterable
in groupby_day the iterable
the generated timestamp 1970-02-27 02:36:36 and value: 100
the generated timestamp 1970-03-0

In [37]:
from scipy import stats
import numpy as np
pts = 1000
np.random.seed(28041990)
a = np.random.normal(0, 1, size=pts)
b = np.random.normal(2, 1, size=pts)
x = np.concatenate((a, b))
k2, p = stats.normaltest(x)
alpha = 1e-3
print("p = {:g}".format(p))
if p < alpha:  # null hypothesis: x comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")

p = 3.27207e-11
The null hypothesis can be rejected
